In [202]:
import pandas as pd
import numpy as np

# 1. 시계열/통계 데이터 (df_time)
np.random.seed(42)
dates = pd.date_range('2023-01-01', periods=100, freq='D')
prices = np.random.lognormal(mean=2, sigma=1, size=100) * 1000 # 왜도가 있는 데이터 생성
sales = np.random.randint(10, 100, size=100).astype(float)

df_time = pd.DataFrame({'Date': dates, 'Price': prices, 'Sales': sales})

# 결측치 강제 생성 (보간법 연습용)
df_time.loc[10:20, 'Sales'] = np.nan 
df_time.loc[50:55, 'Sales'] = np.nan

# 2. 넓은 형태 데이터 (df_wide) - Melt 연습용
data_wide = {
    'Product': ['A', 'B', 'C'],
    '2023-Q1': [100, 150, 200],
    '2023-Q2': [110, 160, 210],
    '2023-Q3': [105, 155, 205],
    '2023-Q4': [120, 170, 220]
}
df = pd.DataFrame(data_wide)

print("데이터 생성 완료!")
print(df_time.head())
print(df.head())

데이터 생성 완료!
        Date         Price  Sales
0 2023-01-01  12142.529844   18.0
1 2023-01-02   6434.896126   71.0
2 2023-01-03  14121.359907   46.0
3 2023-01-04  33886.945659   60.0
4 2023-01-05   5846.520075   53.0
  Product  2023-Q1  2023-Q2  2023-Q3  2023-Q4
0       A      100      110      105      120
1       B      150      160      155      170
2       C      200      210      205      220


In [203]:
skew_orig = df_time['Price'].skew()
#print(skew_orig)

df_time['Price_Log'] = np.log1p(df_time['Price'])
skew_log = df_time['Price_Log'].skew()
result1 = abs(skew_orig - skew_log)
print(round(result1, 2))

2.07


In [204]:
df_time['Sales_fill'] = df_time['Sales'].interpolate(method='linear')
df_time['Prev_Sales'] = df_time['Sales_fill'].shift(1)
result2 = df_time['Sales_fill'].corr(df_time['Prev_Sales'])
print(round(result2, 2))
print(df_time.head())

0.25
        Date         Price  Sales  Price_Log  Sales_fill  Prev_Sales
0 2023-01-01  12142.529844   18.0   9.404552        18.0         NaN
1 2023-01-02   6434.896126   71.0   8.769646        71.0        18.0
2 2023-01-03  14121.359907   46.0   9.555515        46.0        71.0
3 2023-01-04  33886.945659   60.0  10.430815        60.0        46.0
4 2023-01-05   5846.520075   53.0   8.673773        53.0        60.0


In [205]:
#print(df.head())
df_melt = pd.melt(df, id_vars=['Product'], var_name='Quarter', value_name='Revenue')
#print(df_melt)

result3 = df_melt[df_melt['Quarter']=='2023-Q4']['Revenue'].sum()
print(round(result3))

510


In [206]:
min_val = df_time['Price'].min()
max_val = df_time['Price'].max()
df_time['Price_Scaled'] = (df_time['Price'] - min_val) / (max_val - min_val)
result4 = sum(df_time['Price_Scaled'] > 0.5)
print(result4)

8


📝 실전 연습 문제
문제 1. (조건 필터링 & 중앙값 대치) Price 컬럼의 결측치(NaN)를 **City별 Price의 중앙값(median)**으로 채우시오. 그 후, Payment가 'Card'인 데이터들의 Price 평균값을 구하시오. (단, 소수점 셋째 자리에서 반올림하여 둘째 자리까지 출력하시오.)

문제 2. (시계열 & Groupby) Date 컬럼에서 '요일'(Monday=0, ... Sunday=6) 정보를 추출하여 Weekday 컬럼을 만드시오. **'주말' (토요일, 일요일)**에 해당하는 데이터만 필터링한 후, 이 데이터들의 City별 Price **합계(sum)**를 구했을 때 가장 큰 값을 구하시오. (단, 정수로 출력하시오.)

문제 3. (데이터 구간화 & 빈도수) Price 컬럼 값(결측치 처리 완료된 상태 가정)을 기준으로 데이터를 3개의 구간으로 나눕니다.

0 이상 4000 미만: 'Low'

4000 이상 7000 미만: 'Mid'

7000 이상: 'High'

'Mid' 그룹에 속하면서 Product가 'A'인 데이터의 **개수(행의 수)**를 구하시오.

In [207]:
import pandas as pd
import numpy as np

np.random.seed(2024)

# 데이터 생성
dates = pd.date_range('2023-01-01', periods=500, freq='h')
cities = np.random.choice(['Seoul', 'Busan', 'Daegu', 'Incheon'], 500)
products = np.random.choice(['A', 'B', 'C'], 500)
prices = np.random.randint(1000, 10000, 500)
payments = np.random.choice(['Card', 'Cash', 'Pay'], 500)

df = pd.DataFrame({
    'Date': dates,
    'City': cities,
    'Product': products,
    'Price': prices,
    'Payment': payments
})

# 결측치 임의 생성 (Price 컬럼)
df.loc[10:20, 'Price'] = np.nan

# 데이터 확인
print(df.head())

                 Date     City Product   Price Payment
0 2023-01-01 00:00:00    Seoul       A  2725.0    Cash
1 2023-01-01 01:00:00    Daegu       B  9929.0    Card
2 2023-01-01 02:00:00    Seoul       C  3118.0    Card
3 2023-01-01 03:00:00    Seoul       C  2901.0    Card
4 2023-01-01 04:00:00  Incheon       A  2739.0    Cash


In [208]:
# 문제 1
fill_nan = df.groupby('City')['Price'].transform('median')

df['Price'] = df['Price'].fillna(fill_nan)
#print(df.info())
result = df[df['Payment']=='Card']['Price'].mean()
print(round(result, 2))

5426.29


In [209]:
# 문제 2
df['Weekday'] = df['Date'].dt.weekday
q1 = df['Weekday'].isin([5, 6])
result = df.groupby('City')['Price'].sum().sort_values(ascending=False)
print(result)
# loc, iloc, isin 쓰는법


City
Incheon    815941.0
Busan      647059.5
Daegu      641636.0
Seoul      599213.0
Name: Price, dtype: float64


In [210]:
def get_category(price):
    if price < 4000: return 'Low'
    elif price < 7000: return 'Mid'
    else: return 'High'

df['Category'] = df['Price'].apply(get_category)
result = len(df[(df['Category']=='Mid') & (df['Product']=='A')])
print(result)

50


📝 실전 연습 문제 (Large Data)
문제 1. (시계열 분해 & 집계) Date 컬럼을 이용하여 **2023년의 '월별(Month)' 총 범죄 건수(Violent + Theft + Traffic)**를 집계하시오. 그중 총 범죄 건수가 **가장 많았던 월(Month)**의 범죄 건수 합계를 구하시오. (단, 정수로 출력하시오.)

문제 2. (복합 연산 & Groupby 주의점) 전체 기간 동안 각 구(District)별로 데이터가 수집되었습니다. **'Zone_A'**의 연간 10만 명당 평균 강력 범죄(Violent) 발생 건수를 구하시오.

계산 로직: (Zone_A의 전체 Violent 합계 / 데이터 수집 년수) / Zone_A의 인구수 * 100,000

데이터 수집 년수(Years)는 5년(2020~2024)으로 계산합니다.

결과값은 소수점 이하는 버리고 **정수(int)**로 출력하시오.

문제 3. (이상치 탐색 & 조건부 평균) 전체 데이터 중 Traffic 범죄 건수가 상위 1%(quantile(0.99)) 이상인 날들을 **'Traffic_High'**로 정의합니다. 이 'Traffic_High' 그룹에 속하는 데이터들의 Violent 범죄 발생 건수의 평균을 구하시오. (단, 소수점 셋째 자리에서 반올림하여 둘째 자리까지 출력하시오.)

In [211]:
import pandas as pd
import numpy as np

# 랜덤 시드 고정 (정답 통일을 위해)
np.random.seed(2024)

# 1. 기본 설정
n_rows = 50000
districts = [f'Zone_{chr(i)}' for i in range(65, 85)] # Zone_A ~ Zone_T (20개)

# 2. 구별 인구수 매핑 (Zone_A는 30만, Zone_B는 35만... 랜덤 배정)
pop_map = {d: np.random.randint(200000, 800000) for d in districts}

# 3. 데이터 생성
df = pd.DataFrame({
    'Date': np.random.choice(pd.date_range('2020-01-01', '2024-12-31'), n_rows),
    'District': np.random.choice(districts, n_rows),
    'Violent': np.random.randint(0, 15, n_rows),  # 일일 발생 건수라 적게 설정
    'Theft': np.random.randint(0, 30, n_rows),
    'Traffic': np.random.randint(0, 50, n_rows)
})

# 4. 인구수 매핑 및 정렬
df['Population'] = df['District'].map(pop_map)
df = df.sort_values('Date').reset_index(drop=True)

print(f"데이터 생성 완료! 크기: {df.shape}")
print(df.head(10))

데이터 생성 완료! 크기: (50000, 6)
        Date District  Violent  Theft  Traffic  Population
0 2020-01-01   Zone_N        6     21       21      662347
1 2020-01-01   Zone_A       10      3       44      732104
2 2020-01-01   Zone_N        4      8       38      662347
3 2020-01-01   Zone_E       12      0       47      654593
4 2020-01-01   Zone_K       13      5       35      474799
5 2020-01-01   Zone_H        9      5       37      721696
6 2020-01-01   Zone_D        0      7       20      661246
7 2020-01-01   Zone_G       14      5       41      486794
8 2020-01-01   Zone_O       11     28       41      466331
9 2020-01-01   Zone_S        1     10        9      253094


In [212]:
# 문제 1
df['Date'] = pd.to_datetime(df['Date'])
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month

df['Total'] = df['Violent'] + df['Theft'] + df['Traffic']
q1 = df[df['year']==2023]
result = q1.groupby('month')['Total'].sum().sort_values(ascending=False)
print(result.iloc[0])



41536


In [213]:
# 문제 2
q2 = df[df['District']=='Zone_A']
year_ratio = (q2['Violent'].sum() / 5) / q2['Population'].iloc[0] * 100000
print(int(year_ratio))

479


In [214]:
# 문제 3
var = df['Traffic'].quantile(0.99)
def cond(traffic):
    if traffic >= var: return 'Traffic_High'
    else: return 'NoNan'

df['Traffic_check'] = df['Traffic'].apply(cond)

result = df[df['Traffic_check']=='Traffic_High']['Violent'].mean()
print(round(result, 2))

6.94


📝 실전 연습 문제 (약점 보완 & 문자열)
문제 1. (문자열 추출 & 빈도 분석) Product 컬럼에는 대괄호 [ ... ] 안에 제품의 옵션 정보가 들어있습니다. (예: [Black]) 정규표현식을 사용하여 이 옵션 텍스트만 추출(괄호 제외)하여 Option이라는 새 컬럼을 만드세요. 가장 많이 판매된(행 개수 기준) Option의 이름을 출력하시오.

문제 2. (그룹별 결측치 처리 & 평균 계산) Price 컬럼의 결측치를 **Category별 Price의 평균값(mean)**으로 채우세요. (주의: 전체 평균이 아니라, Electronics는 Electronics끼리, Accessories는 Accessories끼리 평균을 구해서 채워야 합니다.) 결측치 처리가 완료된 후, Price의 **표준편차(std)**를 구하시오. (단, 소수점 이하는 버리고 **정수(int)**로 출력하시오.)

문제 3. (시계열 집계 & 이상치 범위 - 지난번 실수 복습) 일별 매출액(Daily_Sales)을 계산하려고 합니다. (매출액 = Price * Quantity, 결측치 처리된 Price 사용) 날짜(Date)별로 총 매출액을 집계한 뒤, 총 매출액이 **상위 10% 이상(>= 0.9 quantile)**인 날짜들을 **'High_Sales_Day'**로 정의합니다. 이 'High_Sales_Day'에 해당하는 날들의 일평균 매출액을 구하시오. (단, 소수점 이하는 버리고 **정수(int)**로 출력하시오.)

In [215]:
import pandas as pd
import numpy as np

np.random.seed(2025)
n_rows = 5000

# 1. 상품명 생성 (대괄호 안에 옵션 포함)
products_base = ['Laptop', 'Phone', 'Monitor', 'Mouse', 'Keyboard']
options = ['[Black]', '[White]', '[Silver]', '[16GB]', '[Wireless]']
categories = ['Electronics', 'Electronics', 'Electronics', 'Accessories', 'Accessories']

prod_list = []
cat_list = []
for _ in range(n_rows):
    idx = np.random.randint(0, 5)
    opt_idx = np.random.randint(0, 5)
    # 상품명 예: "Laptop [Black]"
    prod_name = f"{products_base[idx]} {options[opt_idx]}"
    prod_list.append(prod_name)
    cat_list.append(categories[idx])

# 2. 데이터 생성
df = pd.DataFrame({
    'Date': np.random.choice(pd.date_range('2023-01-01', '2023-12-31'), n_rows),
    'Product': prod_list,
    'Category': cat_list,
    'Price': np.random.randint(50, 2000, n_rows) * 1000, # 50,000 ~ 2,000,000
    'Quantity': np.random.randint(1, 5, n_rows)
})

# 3. 결측치 생성 (Price)
df.loc[np.random.choice(df.index, 500), 'Price'] = np.nan

print(f"데이터 생성 완료! (행: {df.shape[0]})")
print(df.head())

데이터 생성 완료! (행: 5000)
        Date             Product     Category      Price  Quantity
0 2023-08-11  Monitor [Wireless]  Electronics  1030000.0         2
1 2023-12-26       Laptop [16GB]  Electronics        NaN         1
2 2023-10-19    Mouse [Wireless]  Accessories   410000.0         4
3 2023-07-24      Laptop [Black]  Electronics   930000.0         1
4 2023-03-09     Monitor [White]  Electronics  1437000.0         4


In [216]:
# 문제 1
df['Option'] = df['Product'].str.extract(r'\[(.*?)\]')
result = df['Option'].value_counts().sort_values(ascending=False)
print(result.idxmax())

Black


In [217]:
# 문제 2
fill_val = df.groupby('Category')['Price'].transform('mean')

df['Price'] = df['Price'].fillna(fill_val)
result = df['Price'].std()
print(int(result))

533145


In [218]:
# 문제 3
df['Revenue'] = df['Price'] * df['Quantity']
daily_sales = df.groupby('Date')['Revenue'].sum()
#print(daily_sales)

var = daily_sales.quantile(0.9)
high_sales_days = daily_sales[daily_sales >= var]
result = high_sales_days.mean()
print(int(result))

55587015


📝 실전 연습 문제 (데이터셋: Ecommerce_Sales)
문제 1. (문자열 포함 여부 검색) Product 이름에 **'Wireless'**라는 단어가 포함된 상품들만 별도로 분석하려고 합니다. 'Wireless'가 포함된 상품들의 **Price 중앙값(median)**을 구하시오. (단, Price 컬럼의 결측치는 제거(dropna)한 후 계산하고, 결과는 **정수(int)**로 출력하시오.)

문제 2. (시계열 집계 & 정렬) Date 컬럼에서 '월(Month)' 정보를 추출합니다. 'Electronics' 카테고리에 해당하는 데이터만 필터링한 후, **월별 총 매출액(Price * Quantity)**을 집계했을 때, 매출액이 가장 높은 **월(Month)**은 몇 월입니까? (단, 결측치가 있는 행은 매출액 계산 전 제거하고, 정수로 출력하시오.)

문제 3. (순위 구하기 & 인덱싱) 전체 데이터(결측치 제거 후)를 Price(가격)가 높은 순서대로 내림차순 정렬합니다. 이때, 가격이 5번째로 높은 주문의 **Order_ID**는 무엇입니까? (주의: 0부터 시작하는 인덱스 기준이 아니라, 실제 순위 기준 5등입니다. 동점자는 무시하고 행 순서대로 처리합니다.)

In [219]:
print(df.head())

        Date             Product     Category         Price  Quantity  \
0 2023-08-11  Monitor [Wireless]  Electronics  1.030000e+06         2   
1 2023-12-26       Laptop [16GB]  Electronics  1.022305e+06         1   
2 2023-10-19    Mouse [Wireless]  Accessories  4.100000e+05         4   
3 2023-07-24      Laptop [Black]  Electronics  9.300000e+05         1   
4 2023-03-09     Monitor [White]  Electronics  1.437000e+06         4   

     Option       Revenue  
0  Wireless  2.060000e+06  
1      16GB  1.022305e+06  
2  Wireless  1.640000e+06  
3     Black  9.300000e+05  
4     White  5.748000e+06  


In [220]:
# 문제 1
q1 = df.dropna(subset=['Price'])
result = q1[q1['Product'].str.contains('Wireless')]['Price'].median()
print(int(result))

1022305


In [221]:
# 문제 2
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month

q1 = df[df['Category']=='Electronics']
q1['Revenue'] = q1['Price'] * q1['Quantity']
result = q1.groupby('Month')['Revenue'].sum().sort_values(ascending=False)
print(result.idxmax())

9


C:\Users\sangh\AppData\Local\Temp\ipykernel_15296\3975182502.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q1['Revenue'] = q1['Price'] * q1['Quantity']


In [222]:
# 문제 3
df = df.dropna(subset=['Price'])
df = df.sort_values(by='Price', ascending=False)
print(df.iloc[4])

Date        2023-11-20 00:00:00
Product      Monitor [Wireless]
Category            Electronics
Price                 1998000.0
Quantity                      3
Option                 Wireless
Revenue               5994000.0
Month                        11
Name: 2770, dtype: object


📝 실전 연습 문제 (Subway Traffic)

문제 1. (시계열 집계 & 최댓값)Line_2 호선에 해당하는 데이터만 필터링합니다.Date 컬럼을 활용하여 '월(Month)'별 승차 인원(Get_On)의 총합을 구했을 때, 승차 인원이 가장 많았던 **월(Month)**을 구하시오.(단, Get_On의 결측치는 제거하고 계산하며, 정수로 출력하시오.)

문제 2. (파생변수 & 절댓값 정렬)모든 데이터에 대해 '승하차 차이' (Diff) 컬럼을 생성합니다. 계산 식은 Get_On - Get_Off입니다. (결측치가 포함된 행은 제거)이 Diff 값의 **절댓값(Absolute Value)**이 가장 큰 상위 3개 역(Station)을 찾고, 그 중 첫 번째 역의 이름을 출력하시오.(동점자가 있을 경우 행의 순서대로 처리)

문제 3. (IQR 이상치 탐색)Line_1 호선의 Get_Off (하차 인원) 데이터를 대상으로 이상치를 탐색합니다.IQR 방식을 이용하여 이상치(Outlier)로 판단되는 데이터의 개수를 구하시오.$Q1$: 하위 25%, $Q3$: 상위 75%$IQR = Q3 - Q1$이상치 기준: $(Q1 - 1.5 \times IQR)$ 미만 이거나 $(Q3 + 1.5 \times IQR)$ 초과

In [223]:
import pandas as pd
import numpy as np

np.random.seed(2025)
n_rows = 5000

# 데이터 생성
dates = pd.date_range('2023-01-01', '2023-12-31', freq='h') # 시간 단위지만 5000개만 샘플링
dates = np.random.choice(dates, n_rows)

lines = ['Line_1', 'Line_2', 'Line_3', 'Line_4', 'Line_5']
stations = [f'Station_{chr(i)}' for i in range(65, 85)] # A ~ T

df = pd.DataFrame({
    'Date': dates,
    'Line': np.random.choice(lines, n_rows),
    'Station': np.random.choice(stations, n_rows),
    'Get_On': np.random.randint(10, 1000, n_rows),
    'Get_Off': np.random.randint(10, 1000, n_rows)
})

# 결측치 주입 (Get_On)
df.loc[np.random.choice(df.index, 100), 'Get_On'] = np.nan

# 데이터 정렬
df = df.sort_values('Date').reset_index(drop=True)

print("데이터 생성 완료!")
print(df.head())

데이터 생성 완료!
                 Date    Line    Station  Get_On  Get_Off
0 2023-01-01 02:00:00  Line_2  Station_O   292.0      298
1 2023-01-01 05:00:00  Line_4  Station_A   882.0      370
2 2023-01-01 09:00:00  Line_3  Station_S   871.0      813
3 2023-01-01 10:00:00  Line_5  Station_P   636.0      336
4 2023-01-01 11:00:00  Line_2  Station_J   548.0      545


In [224]:
# 문제 1번
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month

df_line2 = df[df['Line']=='Line_2']
q1 = df_line2.groupby('Month')['Get_On'].sum()
print(q1.idxmax())

4


In [225]:
# 문제 2번
df['Diff'] = df['Get_On'] - df['Get_Off']
q2 = df.groupby('Station')['Diff'].sum().sort_values(ascending=False)
print(np.abs(q2).head(3))

Station
Station_H    11493.0
Station_R     8754.0
Station_D     8118.0
Name: Diff, dtype: float64


In [226]:
# 문제 3번
df_1 = df[df['Line']=='Line_1'][['Line', 'Get_Off']]
Q1 = df_1['Get_Off'].quantile(0.25)
Q3 = df_1['Get_Off'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = (Q1 - 1.5 * IQR)
high_bound = (Q3 + 1.5 * IQR)

df_outer = df_1[(df_1['Get_Off'] > high_bound) | (df_1['Get_Off'] < lower_bound)]
result = len(df_outer)
print(result)

#df_2 = df[df['Line']=='Line_1']['Get_Off']
#print(df_2)

0


📝 [제1유형] 대기오염 데이터 분석
문제 1. (이상치 처리 & 조건부 평균) PM10 컬럼에는 센서 오류로 인해 **0보다 작은 값(음수)**이 포함되어 있습니다. 이 **음수 값을 이상치로 간주하여 제거(필터링)**한 데이터를 기준으로, **'City_A'**의 PM10 평균을 구하시오. (단, 소수점 이하는 버리고 **정수(int)**로 출력하시오.)

문제 2. (시계열 추출 & Groupby) Date 컬럼에서 '시간(Hour)' 정보를 추출합니다. 전체 도시를 통틀어, 오전 10시(10)부터 오후 3시(15) 사이(10시, 15시 포함) 시간대의 PM2.5 평균 농도를 구하시오. (단, PM2.5의 결측치는 제거하고 계산하며, 소수점 셋째 자리에서 반올림하여 둘째 자리까지 출력하시오.)

문제 3. (결측치 보간 & 복합 조건) PM2.5 컬럼의 결측치를 **전체 데이터의 중앙값(median)**으로 채웁니다. 그 후, '나쁨' 상태인 데이터를 찾으려고 합니다.

조건 1: PM10이 80 이상

조건 2: PM2.5가 35 이상 위 **두 조건 중 하나라도 만족(OR)**하는 데이터의 **행 개수(row count)**를 구하시오. (단, 문제 1번에서 언급한 PM10의 음수 이상치는 미리 제거된 상태에서 진행하시오.)

In [255]:
import pandas as pd
import numpy as np

np.random.seed(2025)
n_rows = 2000

# 1. 데이터 생성
dates = pd.date_range('2023-01-01', periods=n_rows, freq='h')
cities = np.random.choice(['City_A', 'City_B', 'City_C', 'City_D'], n_rows)

# PM10: 평균 50, 표준편차 20 (가끔 센서 오류로 -999 값 발생)
pm10 = np.random.normal(50, 20, n_rows)
error_indices = np.random.choice(range(n_rows), 20, replace=False)
pm10[error_indices] = -999 # 센서 오류 값

# PM2.5: PM10의 약 60% 수준 (결측치 발생)
pm25 = pm10 * 0.6 + np.random.normal(0, 5, n_rows)
nan_indices = np.random.choice(range(n_rows), 50, replace=False)
pm25[nan_indices] = np.nan # 결측치

df = pd.DataFrame({
    'Date': dates,
    'City': cities,
    'PM10': pm10,
    'PM2.5': pm25,
    'Temp': np.random.normal(15, 10, n_rows)
})

print("데이터 생성 완료!")
print(df.head())

데이터 생성 완료!
                 Date    City       PM10      PM2.5       Temp
0 2023-01-01 00:00:00  City_C  41.395782  25.009453  28.385131
1 2023-01-01 01:00:00  City_C  49.862120  25.841500  -3.906242
2 2023-01-01 02:00:00  City_C  36.678939  18.679058  35.055428
3 2023-01-01 03:00:00  City_C  66.587579  40.386310  10.693343
4 2023-01-01 04:00:00  City_A  45.675755  25.241757   6.889414


In [263]:
# 문제 1번
#df.loc[df['PM10'] < 0, 'PM10'] = 0
q1 = df[df['PM10'] >= 0]
q2 = q1.groupby('City')['PM10'].mean()
print(int(q2.loc['City_A']))


51


In [257]:
# 문제 2번
df['Date'] = pd.to_datetime(df['Date'])
df['Hour'] = df['Date'].dt.hour
df_t = df[(df['Hour'] >= 10) & (df['Hour'] <= 15)]
result = df_t['PM2.5'].mean()
print(round(result, 2))

23.78


In [262]:
# 문제 3번
df = df[df['PM10'] >= 0]

#print(df['PM2.5'].isna().sum())
med = df['PM2.5'].median()
df['PM2.5'] = df['PM2.5'].fillna(med)

df_filter = df[(df['PM10'] >= 80) | (df['PM2.5'] >= 35)]
print(len(df_filter))


706


📝 [제1유형] 쇼핑몰 배송/판매 분석
문제 1. (순위 구하기 - Rank) 'Tech' 카테고리의 데이터만 추출합니다. 매출액(Sales)이 높은 순서대로 **순위(Rank)**를 매길 때, **상위 10위 (1등~10등)**에 해당하는 데이터들의 Sales 평균을 구하시오. (단, 동점자가 있을 경우 'min' 방식(예: 1등이 2명이면 둘 다 1등, 다음은 3등)으로 순위를 매기며, 결과는 정수(int)로 출력하시오.)

문제 2. (날짜 차이 계산 - Timedelta) 주문일(Order_Date)로부터 배송일(Ship_Date)까지 걸린 기간을 **'Delivery_Days'**라고 합니다. (Ship_Date - Order_Date) 전체 데이터 중 Delivery_Days가 **3일 이내(<= 3일)**인 데이터의 비율(%)을 구하시오. (단, 결과는 백분율로 변환하지 말고 0~1 사이 비율 그대로 소수점 셋째 자리에서 반올림하여 둘째 자리까지 출력하시오.)

문제 3. (그룹별 변동폭 - Diff) 각 상품(Product)별로 주문 날짜(Order_Date) 순서대로 정렬했을 때, 이전 판매량(Quantity) 대비 현재 판매량의 **차이(절댓값)**가 가장 컸던 값을 찾으시오. 즉, 모든 상품을 통틀어 **"전번 주문 대비 판매량 변동폭"**의 최댓값을 구하시오. (단, 각 상품의 첫 번째 주문은 이전 데이터가 없으므로 무시합니다.)

In [264]:
import pandas as pd
import numpy as np

np.random.seed(2025)
n_rows = 1000

# 1. 데이터 생성
order_dates = pd.date_range('2023-01-01', '2023-12-31', freq='D')
order_dates = np.random.choice(order_dates, n_rows)

# 배송일은 주문일로부터 1~7일 후
ship_days = np.random.randint(1, 8, n_rows)
ship_dates = [pd.to_datetime(d) + pd.Timedelta(days=i) for d, i in zip(order_dates, ship_days)]

products = [f'Prod_{i}' for i in range(1, 11)] # 상품 10개

df = pd.DataFrame({
    'Order_Date': order_dates,
    'Ship_Date': ship_dates,
    'Category': np.random.choice(['Furniture', 'Office', 'Tech'], n_rows),
    'Product': np.random.choice(products, n_rows),
    'Sales': np.random.randint(100, 10000, n_rows) * 10,
    'Quantity': np.random.randint(1, 10, n_rows)
})

# 날짜 정렬
df = df.sort_values('Order_Date').reset_index(drop=True)

print("데이터 생성 완료!")
print(df.head())

데이터 생성 완료!
  Order_Date  Ship_Date   Category Product  Sales  Quantity
0 2023-01-01 2023-01-03       Tech  Prod_8  41930         4
1 2023-01-01 2023-01-06       Tech  Prod_5  31530         2
2 2023-01-01 2023-01-07  Furniture  Prod_8  65670         5
3 2023-01-03 2023-01-07     Office  Prod_1  88220         8
4 2023-01-03 2023-01-05     Office  Prod_5  94630         8


In [281]:
# 문제 1번
df_tech = df[df['Category']=='Tech'].sort_values(by='Sales', ascending=False)
print(int(df_tech['Sales'].head(10).mean()))

# 정석 풀이
df_tech = df[df['Category']=='Tech'].copy()
df_tech['Rank'] = df_tech['Sales'].rank(method='min', ascending=False)
result = df_tech[df_tech['Rank'] <= 10]['Sales'].mean()
print(int(result))


97513
97513


In [280]:
# 문제 2번
df['Order_Date'] = pd.to_datetime(df['Order_Date'])
df['Ship_Date'] = pd.to_datetime(df['Ship_Date'])
df['Delivery_Days'] = (df['Ship_Date'] - df['Order_Date']).dt.days

inner_len = len(df[df['Delivery_Days'] <= 3])
df_len = len(df)
result = inner_len / df_len
print(round(result, 2))


0.44


In [282]:
# 문제 3번
df = df.sort_values(by=['Product', 'Order_Date'])
df['Vol_Diff'] = df.groupby('Product')['Quantity'].diff()
df['Vol_Diff_Abs'] = df['Vol_Diff'].abs()
print(int(df['Vol_Diff_Abs'].max()))

8


📝 [제2유형] 은행 고객 이탈 예측
문제: 은행 고객 데이터(train)를 이용하여 고객의 이탈 여부(Exited)를 예측하는 모델을 만들고, 제공된 평가용 데이터(test)에 대한 예측 결과를 csv 파일로 제출하시오.

1. 데이터 설명

Target: Exited (1: 이탈, 0: 유지)

Features:

CustomerId: 고객 ID (식별자)

Surname: 성 (String)

CreditScore: 신용 점수

Geography: 국가 (France, Germany, Spain)

Gender: 성별

Age, Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary

2. 제출 형식

result.csv 파일로 저장해야 합니다.

반드시 **CustomerId**와 Exited 두 개의 컬럼만 포함해야 합니다.

Exited 컬럼에는 **이탈할 확률(Probability)**을 제출하시오. (0과 1 사이의 소수)

3. 평가 지표

ROC-AUC (값이 클수록 좋음)

In [229]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(2025)
n_rows = 3000

# 1. 데이터 생성
data = {
    'CustomerId': np.arange(10001, 10001 + n_rows),
    'Surname': np.random.choice(['Smith', 'Johnson', 'Williams', 'Brown', 'Jones'], n_rows),
    'CreditScore': np.random.randint(350, 850, n_rows),
    'Geography': np.random.choice(['France', 'Germany', 'Spain'], n_rows),
    'Gender': np.random.choice(['Male', 'Female'], n_rows),
    'Age': np.random.randint(18, 92, n_rows),
    'Tenure': np.random.randint(0, 10, n_rows),
    'Balance': np.random.uniform(0, 250000, n_rows),
    'NumOfProducts': np.random.randint(1, 4, n_rows),
    'HasCrCard': np.random.randint(0, 2, n_rows),
    'IsActiveMember': np.random.randint(0, 2, n_rows),
    'EstimatedSalary': np.random.uniform(11, 200000, n_rows),
    'Exited': np.random.randint(0, 2, n_rows) # Target
}

df = pd.DataFrame(data)

# 2. 결측치 및 이상치 강제 주입 (전처리 연습용)
# Age 결측치
df.loc[np.random.choice(df.index, 50), 'Age'] = np.nan
# Balance 결측치
df.loc[np.random.choice(df.index, 30), 'Balance'] = np.nan

# 3. Train / Test 분리 (시험 환경 흉내)
X = df.drop('Exited', axis=1)
y = df['Exited']

# 실제 시험에서는 X_test에 정답(y)이 없습니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 데이터프레임 합치기 (시험에서 제공되는 형태)
train = pd.concat([X_train, y_train], axis=1)
test = X_test # y_test는 숨겨져 있음 (채점용)

# CSV 파일로 저장되었다고 가정하고 변수명만 사용합니다.
print(f"데이터 생성 완료!")
print(f"train shape: {train.shape}")
print(f"test shape: {test.shape}")
print(df.head())

데이터 생성 완료!
train shape: (2100, 13)
test shape: (900, 12)
   CustomerId   Surname  CreditScore Geography  Gender   Age  Tenure  \
0       10001  Williams          423    France    Male  43.0       5   
1       10002     Jones          532   Germany  Female  54.0       9   
2       10003     Smith          470     Spain    Male  75.0       8   
3       10004     Brown          627   Germany  Female  81.0       0   
4       10005     Brown          520     Spain    Male  67.0       5   

         Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  \
0  156207.782854              3          0               0     61837.658280   
1  149914.101471              2          1               1     54490.748664   
2  132768.028528              3          0               1    126846.371424   
3   44238.586796              1          0               0    172944.034041   
4  105158.756047              2          1               1     58251.970535   

   Exited  
0       0  
1       0  

In [230]:
#print(train.info())
# 결측치: Age, Balance
# object: Geography, Gender

#print(test.info())
# 결측치: Age, Balance

train['Age'] = train['Age'].fillna(train['Age'].mean())
test['Age'] = test['Age'].fillna(train['Age'].mean())

train['Balance'] = train['Balance'].fillna(train['Balance'].mean())
test['Balance'] = test['Balance'].fillna(train['Balance'].mean())

#print(test.info())

X = train.drop(['CustomerId', 'Surname', 'Exited'], axis=1)
y = train['Exited']
X_submit = test.drop(['CustomerId', 'Surname'], axis=1)

cols = ['Geography', 'Gender']

X = pd.get_dummies(X, columns=cols)
X_submit = pd.get_dummies(X_submit, columns=cols)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier()
model.fit(X_train, y_train)
pred_proba = model.predict_proba(X_val)[:, 1]

from sklearn.metrics import roc_auc_score
roc = roc_auc_score(y_val, pred_proba)
print(round(roc, 3))

model.fit(X, y)
final_pred = model.predict(X_submit)

result = pd.DataFrame({
    'Id': test['CustomerId'],
    'pred': final_pred
})
result.to_csv('result.csv', index=False)
print(pd.read_csv('result.csv').head())



0.532
      Id  pred
0  11802     1
1  11191     1
2  11818     1
3  10252     1
4  12506     1


📝 [제2유형] 자전거 대여량 예측 (회귀)
문제: 제공된 자전거 대여 데이터(train)를 이용하여 대여 횟수(count)를 예측하는 모델을 만들고, 평가용 데이터(test)에 대한 예측 결과를 csv 파일로 제출하시오.

1. 데이터 설명

Target: count (대여 횟수, 수치형)

Features:

ID: 고유 식별자

Date: 날짜 (datetime)

Temperature, Humidity, Windspeed, Visibility: 날씨 정보

Holiday: 공휴일 여부 (Yes/No)

Hour: 시간 (0~23)

2. 제출 형식

result.csv 파일로 저장해야 합니다.

**ID**와 pred 두 개의 컬럼만 존재해야 합니다.

pred 컬럼에는 예측한 대여 횟수(실수 또는 정수)를 입력하시오.

3. 평가 지표

MAE (Mean Absolute Error)

(값이 작을수록 좋은 성능입니다.)

In [231]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(2025)
n_rows = 2000

# 1. 데이터 생성
data = {
    'ID': np.arange(1000, 1000 + n_rows),
    'Date': pd.date_range('2023-01-01', periods=n_rows, freq='h'),
    'Temperature': np.random.normal(20, 5, n_rows), # 기온
    'Humidity': np.random.randint(30, 90, n_rows),  # 습도
    'Windspeed': np.random.uniform(0, 10, n_rows),  # 풍속
    'Visibility': np.random.randint(500, 2000, n_rows), # 가시거리
    'Holiday': np.random.choice(['No', 'Yes'], n_rows, p=[0.95, 0.05]), # 공휴일
    'Hour': np.random.randint(0, 24, n_rows), # 시간
    'count': 0 # Target 초기화
}

df = pd.DataFrame(data)

# Target 생성 (대여량은 기온이 높고, 퇴근시간에 많다는 가정)
df['count'] = 100 + (df['Temperature'] * 3) + (df['Hour'] * 5) - (df['Humidity'] * 0.5)
df['count'] = df['count'] + np.random.normal(0, 20, n_rows) # 노이즈 추가
df['count'] = df['count'].apply(lambda x: max(0, int(x))) # 음수 방지 및 정수화

# 2. 결측치 주입 (전처리 연습용)
df.loc[np.random.choice(df.index, 50), 'Windspeed'] = np.nan

# 3. Train / Test 분리
X = df.drop('count', axis=1)
y = df['count']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

train = pd.concat([X_train, y_train], axis=1)
test = X_test

print(f"데이터 생성 완료!")
print(f"train shape: {train.shape}")
print(f"test shape: {test.shape}")
print(train.head())

데이터 생성 완료!
train shape: (1400, 9)
test shape: (600, 8)
        ID                Date  Temperature  Humidity  Windspeed  Visibility  \
836   1836 2023-02-04 20:00:00    27.741598        39   4.947782         836   
575   1575 2023-01-24 23:00:00    20.760819        80        NaN        1787   
557   1557 2023-01-24 05:00:00    15.534825        61   2.401177        1261   
1235  2235 2023-02-21 11:00:00    26.003205        64   7.495187         971   
1360  2360 2023-02-26 16:00:00    18.989828        52   0.973485         958   

     Holiday  Hour  count  
836       No    14    205  
575       No     5    153  
557       No    10    204  
1235      No     7    152  
1360      No     8    138  


In [232]:
#print(test.info())
# 결측치: windspeed
# 인코딩: Holiday
# 삭제: ID, Date

train['Windspeed'] = train['Windspeed'].fillna(train['Windspeed'].mean())
test['Windspeed'] = test['Windspeed'].fillna(train['Windspeed'].mean())

X = train.drop(['ID', 'Date', 'count'], axis=1)
y = train['count']
X_submit = test.drop(['ID', 'Date'], axis=1)

X = pd.get_dummies(X, columns=['Holiday'])
X_submit = pd.get_dummies(X_submit, columns=['Holiday'])

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_val)

from sklearn.metrics import mean_absolute_error
pred_val = mean_absolute_error(y_val, pred)
print(round(pred_val, 3))

model.fit(X, y)
final_pred = model.predict(X_submit)

result = pd.DataFrame({
    'ID': test['ID'],
    'pred': final_pred
})
result.to_csv('result1.csv', index=False)
print(pd.read_csv('result1.csv').head())

17.27
     ID    pred
0  2860  127.66
1  1353  118.44
2  2333  182.31
3  1905  221.30
4  2289  205.26


📝 [제2유형] 전자상거래 구매 예측
문제: 제공된 고객 데이터(train)를 이용하여 고객이 상품을 구매할지(Purchase) 여부를 예측하는 모델을 만들고, 평가용 데이터(test)에 대한 예측 결과를 result.csv 파일로 제출하시오.

1. 데이터 설명

Target: Purchase (1: 구매, 0: 비구매)

Features:

UserID: 고객 ID (불필요한 식별자)

Age, AnnualIncome, BrowsingTime, PagesViewed: 수치형 정보

MembershipTier: 회원 등급 (Bronze, Silver, Gold)

DeviceType: 접속 기기 (Mobile, Desktop, Tablet)

2. 제출 형식

result.csv 파일로 저장.

**UserID**와 pred 두 개의 컬럼만 포함.

pred 컬럼에는 **구매할 확률(Probability)**을 제출하시오. (0~1 사이 값)

3. 평가 지표

ROC-AUC

In [233]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(2025)
n_rows = 4000

# 1. 데이터 생성
data = {
    'UserID': np.arange(10001, 10001 + n_rows),
    'Age': np.random.randint(18, 70, n_rows),
    'AnnualIncome': np.random.randint(20000, 150000, n_rows),
    'BrowsingTime': np.random.normal(30, 10, n_rows), # 분 단위
    'PagesViewed': np.random.randint(1, 20, n_rows),
    'MembershipTier': np.random.choice(['Bronze', 'Silver', 'Gold'], n_rows, p=[0.5, 0.3, 0.2]),
    'DeviceType': np.random.choice(['Mobile', 'Desktop', 'Tablet'], n_rows),
    'Purchase': 0 # Target 초기화
}

df = pd.DataFrame(data)

# 2. Target 생성 (구매 확률: 골드 회원일수록, 수입이 많을수록, 많이 볼수록 높음)
logits = -5 + (df['AnnualIncome'] / 50000) + (df['BrowsingTime'] / 20) + (df['PagesViewed'] / 5)
logits += np.where(df['MembershipTier'] == 'Gold', 2, 0)
logits += np.where(df['MembershipTier'] == 'Silver', 1, 0)
probs = 1 / (1 + np.exp(-logits))
df['Purchase'] = np.random.binomial(1, probs)

# 3. 데이터 분리
X = df.drop('Purchase', axis=1)
y = df['Purchase']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 시험 제공 형태
train = pd.concat([X_train, y_train], axis=1)
test = X_test

print("데이터 생성 완료! (결측치 없음)")
print(f"train shape: {train.shape}")
print(f"test shape: {test.shape}")

데이터 생성 완료! (결측치 없음)
train shape: (2800, 8)
test shape: (1200, 7)


In [234]:
#print(test.info())
# 제거: UserID
# 인코딩: MembershipTier, DeviceType
# 확률 -> classfier

X = train.drop(['UserID', 'Purchase'], axis=1)
y = train['Purchase']
X_submit = test.drop('UserID', axis=1)

cols = ['MembershipTier', 'DeviceType']

X = pd.get_dummies(X, columns=cols)
X_submit = pd.get_dummies(X_submit, columns=cols)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
pred_proba = model.predict_proba(X_val)[:,1]

from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_val, pred_proba)
print(round(score, 3))

model.fit(X, y)
pred_proba_final = model.predict_proba(X_submit)[:,1]

result = pd.DataFrame({
    'UserID': test['UserID'],
    'pred': pred_proba_final
})

result.to_csv('result2.csv', index=False)
print(pd.read_csv('result2.csv'))

0.806
      UserID  pred
0      10556  0.51
1      13492  0.71
2      10528  0.99
3      13926  0.62
4      12990  0.95
...      ...   ...
1195   13857  0.73
1196   10227  0.50
1197   11613  0.91
1198   10536  0.64
1199   13849  0.66

[1200 rows x 2 columns]


📝 [제3유형] 통계적 가설 검정
문제 1. (대응표본 t-검정) 'Treatment'(투약군) 그룹에 속한 환자들의 **투약 전(BP_Pre)**과 투약 후(BP_Post) 혈압 차이가 유의미한지 검정하려고 합니다. (가설: 투약 후 혈압이 투약 전과 다르다.) **대응표본 t-검정(Paired t-test)**을 수행하고, **검정통계량(t-value)**을 구하시오. (단, 반올림하여 소수 셋째 자리까지 출력하시오.)

문제 2. (카이제곱 독립성 검정) 투여 그룹(Group)과 부작용 발생 여부(Side_Effects)가 서로 관련이 있는지(독립이 아닌지) 검정하려고 합니다. 카이제곱 독립성 검정을 수행하고, **p-값(p-value)**을 구하시오. (단, 반올림하여 소수 넷째 자리까지 출력하시오.)

문제 3. (로지스틱 회귀분석 & 오즈비) 치료 성공 여부(Success)를 종속변수로 하고, Age와 BMI를 독립변수로 하는 로지스틱 회귀분석을 수행하시오. 이때 **BMI 변수의 오즈비(Odds Ratio)**를 구하시오. (단, statsmodels를 사용하고, 결과값은 반올림하여 소수 셋째 자리까지 출력하시오.)

In [235]:
import pandas as pd
import numpy as np

np.random.seed(2025)
n = 300

data = {
    'ID': range(1, n + 1),
    'Group': np.random.choice(['Treatment', 'Control'], n),
    'Age': np.random.randint(25, 80, n),
    'BMI': np.random.normal(25, 4, n),
    'BP_Pre': np.random.normal(130, 10, n),
    'Side_Effects': np.random.choice(['Yes', 'No'], n, p=[0.15, 0.85])
}

df = pd.DataFrame(data)

# 투약 후 혈압 생성 (Treatment 그룹만 혈압이 떨어지는 효과 부여)
def get_bp_post(row):
    change = np.random.normal(-5, 5) if row['Group'] == 'Treatment' else np.random.normal(0, 3)
    return row['BP_Pre'] + change

df['BP_Post'] = df.apply(get_bp_post, axis=1)

# 치료 성공 여부 (BMI가 낮을수록, 나이가 적을수록 성공 확률 높음)
logits = 2 - 0.1 * df['BMI'] - 0.02 * df['Age'] + np.random.normal(0, 1, n)
probs = 1 / (1 + np.exp(-logits))
df['Success'] = np.random.binomial(1, probs)

print("데이터 생성 완료!")
print(df.head())

데이터 생성 완료!
   ID      Group  Age        BMI      BP_Pre Side_Effects     BP_Post  Success
0   1  Treatment   49  25.536214  115.545523          Yes  107.602568        0
1   2  Treatment   75  25.708310  118.339493           No  114.446628        0
2   3  Treatment   45  25.510836  115.130393           No  114.998380        0
3   4  Treatment   29  27.439367  128.171946           No  125.530657        0
4   5  Treatment   32  19.326160  129.338332           No  119.815970        0


In [236]:
# 문제 1
target_df = df[df['Group']=='Treatment']
before = target_df['BP_Pre']
after = target_df['BP_Post']

from scipy.stats import ttest_rel
t_stat, p_val = ttest_rel(before, after)
print(round(t_stat, 3))

11.205


In [237]:
# 문제 2
from scipy.stats import chi2_contingency
group = df['Group']
side_effects = df['Side_Effects']

ct = pd.crosstab(group, side_effects)

chi2, p_val, ddof, expected = chi2_contingency(ct)
print(round(p_val, 3))

0.283


In [238]:
# 문제 3
from statsmodels.formula.api import logit
model = logit('Success ~ Age + BMI', data=df).fit()
#print(model.summary())

import numpy as np
odds_ratio = np.exp(model.params['BMI'])
print(round(odds_ratio, 3))

Optimization terminated successfully.
         Current function value: 0.465692
         Iterations 6
0.939


📝 [제3유형] 회귀분석 종합 문제
문제 1. (다중회귀분석 - OLS) 투약 후 혈압(BP_Post)을 종속변수로 하고, 투약 전 혈압(BP_Pre), 나이(Age), 체질량지수(BMI)를 독립변수로 하는 다중선형회귀 모델을 생성하시오. 이때, **BP_Pre (투약 전 혈압)의 회귀계수(Coefficient)**를 구하시오. (단, 소수점 넷째 자리에서 반올림하여 셋째 자리까지 출력하시오.)

문제 2. (로지스틱 회귀 - 범주형 변수의 오즈비) 치료 성공 여부(Success)를 종속변수로 하고, Age, BMI, Group을 독립변수로 하는 로지스틱 회귀 모델을 생성하시오. 이때, 'Treatment'(투약군) 그룹의 환자는 'Control'(대조군) 그룹의 환자보다 치료 성공 오즈(Odds)가 몇 배인지(오즈비) 구하시오. (단, Control을 기준(0), Treatment를 비교(1)로 처리하며, 오즈비는 소수점 넷째 자리에서 반올림하여 셋째 자리까지 출력하시오.)

문제 3. (로지스틱 회귀 - 확률 예측) 2번 문제에서 만든 로지스틱 회귀 모델을 사용하여, 전체 데이터(df) 환자들의 **치료 성공 확률(Probability)**을 예측하시오. 예측된 치료 성공 확률이 **0.6 이상(>= 0.6)**인 환자의 **인원수(명)**를 구하시오.

In [239]:
import pandas as pd
import numpy as np

np.random.seed(2025)
n = 500

# 데이터 생성
data = {
    'ID': range(1, n + 1),
    'Group': np.random.choice(['Treatment', 'Control'], n), # 투약군, 대조군
    'Age': np.random.randint(25, 80, n),
    'BMI': np.random.normal(25, 4, n),
    'BP_Pre': np.random.normal(130, 10, n), # 투약 전 혈압
}

df = pd.DataFrame(data)

# 종속변수 생성 (선형 관계)
# 투약 후 혈압 = 투약 전 * 0.9 + 나이 * 0.1 + BMI * 0.5 - (Treatment인 경우 10)
def get_bp_post(row):
    effect = -10 if row['Group'] == 'Treatment' else 0
    base = row['BP_Pre'] * 0.9 + row['Age'] * 0.1 + row['BMI'] * 0.5 + 10
    return base + effect + np.random.normal(0, 2, 1)[0]

df['BP_Post'] = df.apply(get_bp_post, axis=1)

# 종속변수 생성 (로지스틱 관계)
# Treatment일 때 성공 확률 증가, BMI 낮을수록 성공
df['Treatment_Code'] = df['Group'].map({'Treatment': 1, 'Control': 0})
logits = -2 + 1.5 * df['Treatment_Code'] - 0.1 * df['BMI'] + 0.02 * df['Age']
probs = 1 / (1 + np.exp(-logits))
df['Success'] = np.random.binomial(1, probs)

print("데이터 생성 완료!")
print(df.head(3))

데이터 생성 완료!
   ID      Group  Age        BMI      BP_Pre     BP_Post  Treatment_Code  \
0   1  Treatment   39  30.087421  128.659704  135.787762               1   
1   2  Treatment   75  27.065314  128.106733  136.317723               1   
2   3  Treatment   73  25.347528  138.199694  144.206251               1   

   Success  
0        0  
1        1  
2        1  


In [240]:
# 1번 문제
from statsmodels.formula.api import ols
model = ols('BP_Post ~ BP_Pre + Age + BMI', data=df).fit()
coef = model.params['BP_Pre']
print(round(coef, 3))

0.895


In [241]:
# 2번 문제
from statsmodels.formula.api import logit
model = logit('Success ~ Age + BMI + Group', data=df).fit()
coef = model.params['Group[T.Treatment]']
# 설명 (자동 더미 변수화 - 문자열)
odds_ratio = np.exp(coef)
print(round(odds_ratio, 3))

Optimization terminated successfully.
         Current function value: 0.208368
         Iterations 9
19.577


In [242]:
# 3번 문제 - 외우기
pred_proba = model.predict(df)
target = pred_proba[pred_proba >= 0.6]
result = len(target)
print(result)

0


📝 [제3유형] 분산분석표 (ANOVA Table) 해석
문제 1. (ANOVA 모델 적합) 식이요법 유형(Diet)에 따른 체중 감량(Weight_Loss)의 차이를 분석하기 위해, ols 함수를 사용하여 선형 회귀 모델을 생성하고 적합(fit)시키시오. 생성된 모델 객체를 이용하여 anova_lm 함수를 실행하고, 결과 테이블에서 **Diet 변수의 F-통계량(F-value)**을 구하시오. (단, 반올림하여 소수 셋째 자리까지 출력하시오.)

문제 2. (잔차 제곱합 - SSE) 1번에서 구한 분산분석표(ANOVA Table)를 확인하여, 설명되지 않는 변동인 **잔차(Residual)의 제곱합(Sum of Squares)**을 구하시오. (단, 반올림하여 소수 셋째 자리까지 출력하시오.)

문제 3. (p-value 해석) 분산분석 결과의 **p-값(PR(>F))**을 구하고, 유의수준 0.05 하에서 귀무가설(세 그룹의 평균은 같다)을 기각할 수 있는지 판단하시오. (p-값은 소수 넷째 자리까지 반올림하여 출력하고, 기각 여부는 "기각" 또는 "채택"으로 출력하시오. 정답 예시: 0.0123 기각)

In [243]:
import pandas as pd
import numpy as np

np.random.seed(2025)
n = 150

# 데이터 생성
data = {
    'ID': range(1, n + 1),
    'Diet': np.random.choice(['Type_A', 'Type_B', 'Type_C'], n),
    'Weight_Loss': np.zeros(n)
}

df = pd.DataFrame(data)

# 그룹별 효과 부여 (C > B > A 순으로 감량 효과 큼)
def get_weight_loss(diet):
    base = np.random.normal(2, 1) # 기본 감량
    if diet == 'Type_B': return base + 2  # B는 +2kg 더 감량
    elif diet == 'Type_C': return base + 4 # C는 +4kg 더 감량
    else: return base # A는 기본

df['Weight_Loss'] = df['Diet'].apply(get_weight_loss)

print("데이터 생성 완료!")
print(df.head())

데이터 생성 완료!
   ID    Diet  Weight_Loss
0   1  Type_C     6.616633
1   2  Type_C     5.644096
2   3  Type_C     6.007296
3   4  Type_C     3.862329
4   5  Type_A     2.226040


In [244]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
model = ols('Weight_Loss ~C(Diet)', data=df).fit()
anova_table = anova_lm(model)
print(anova_table)

f_stat = anova_table.loc['C(Diet)', 'F']
print(round(f_stat, 3))

sse = anova_table.loc['Residual', 'sum_sq']
print(round(sse, 3))

p_val = anova_table.loc['C(Diet)', 'PR(>F)']
print(round(p_val, 4))

             df      sum_sq     mean_sq          F        PR(>F)
C(Diet)     2.0  433.580080  216.790040  238.35233  7.359841e-47
Residual  147.0  133.701801    0.909536        NaN           NaN
238.352
133.702
0.0


📝 [제3유형] 비모수 & 비율 & 회귀 평가

문제 1. (비모수 대응표본 검정 - 윌콕슨 부호순위)동일한 고객의 캠페인 전(Satis_Pre)과 후(Satis_Post)의 만족도 차이를 비교하려고 합니다.데이터가 정규성을 만족하지 않는다고 가정하고, **윌콕슨 부호순위 검정(Wilcoxon Signed-Rank Test)**을 수행하시오.이때의 **검정통계량(statistic)**을 구하시오.(단, 반올림하여 소수 셋째 자리까지 출력하시오.)

문제 2. (선형 회귀분석 - 결정계수 $R^2$)구매 금액(Purchase)을 종속변수로 하고, 캠페인 전 만족도(Satis_Pre)와 후 만족도(Satis_Post)를 독립변수로 하는 다중선형회귀 모델을 생성하시오. (ols 사용)이 모델의 결정계수(R-squared) 값을 구하시오.(단, 반올림하여 소수 셋째 자리까지 출력하시오.)
       
문제 3. (비율 검정 - 1표본)전체 고객 중 이벤트 배너를 클릭(Clicked=1)한 비율이 **35%**와 다른지 검정하려고 합니다.**1표본 비율 검정(One-Proportion Z-test)**을 수행하고 p-값을 구하시오.(단, proportions_ztest를 사용하되, alternative='two-sided'(양측검정)로 설정하며, p-값은 소수 넷째 자리까지 출력하시오.)

In [245]:
import pandas as pd
import numpy as np

np.random.seed(2025)
n = 200

data = {
    'ID': range(1, n + 1),
    'Store': np.random.choice(['Gangnam', 'Hongdae', 'Jamsil'], n),
    'Satis_Pre': np.random.randint(3, 8, n),
    'Purchase': np.random.normal(50000, 10000, n),
    'Clicked': np.random.binomial(1, 0.4, n) # 40% 확률로 클릭
}

df = pd.DataFrame(data)

# 만족도 후: 전보다 평균 1.5점 상승, 정규분포 아님(순위형)
df['Satis_Post'] = df['Satis_Pre'] + np.random.choice([0, 1, 2, 3], n, p=[0.1, 0.3, 0.4, 0.2])
df['Satis_Post'] = df['Satis_Post'].clip(upper=10) # 10점 만점 제한

print("데이터 생성 완료!")
print(df.head())

데이터 생성 완료!
   ID    Store  Satis_Pre      Purchase  Clicked  Satis_Post
0   1   Jamsil          4  50280.810091        1           6
1   2   Jamsil          5  46781.064946        1           5
2   3   Jamsil          7  37092.167520        1           9
3   4   Jamsil          6  65087.481495        1           8
4   5  Gangnam          7  58943.399478        1           9


In [246]:
# 문제 1번
group_before = df['Satis_Pre']
group_after = df['Satis_Post']

from scipy.stats import wilcoxon
stat, p_val = wilcoxon(group_before, group_after)
print(round(stat, 3))

0.0


In [247]:
# 문제 2번
from statsmodels.formula.api import ols
model = ols('Purchase ~ Satis_Pre + Satis_Post', data=df).fit()
print(round(model.rsquared, 3))

0.002


In [248]:
# 문제 3번
count = df['Clicked'].sum()
nobs = len(df)

from statsmodels.stats.proportion import proportions_ztest
stat, p_val = proportions_ztest(count, nobs, value=0.35, alternative='two-sided')
print(round(p_val, 4))

0.8825


📝 [제3유형] 통신사 고객 이탈 분석
한 통신 회사에서는 고객 이탈을 줄이고자 주요 요인들을 분석하기로 결정하였다. 이를 위해 500명의 고객 데이터를 수집했으며 고객의 서비스 이용 및 가입정보, 그리고 일부 개인적인 속성을 조사하였다.

1. 모든 컬럼(ID 제외)을 활용하여 Churn(이탈 여부)을 예측하는 로지스틱 회귀 분석을 시행한 후, col1 칼럼의 p-value를 구하시오. (단, Contract 변수는 범주형으로 처리하며, 결과값은 반올림하여 소수 셋째 자리까지 출력하시오.)

2. 위 모델에서 PhoneService를 받지 않은 고객(0) 대비 받은 고객(1)의 이탈 확률 오즈비(Odds Ratio)를 구하시오. (단, 반올림하여 소수 셋째 자리까지 출력하시오.)

3. 위 모델을 활용하여 전체 고객의 이탈 확률(Probability)을 예측하고, 이탈할 확률이 0.3 이상인 고객 수를 구하시오.

In [249]:
import pandas as pd
import numpy as np

np.random.seed(2025)
n = 500

# 1. 독립변수 생성
data = {
    'ID': range(1, n + 1),
    'col1': np.random.normal(70, 30, n), # 월 요금 (수치형)
    'Tenure': np.random.randint(1, 72, n), # 가입 기간 (수치형)
    'PhoneService': np.random.choice([0, 1], n, p=[0.1, 0.9]), # 0: 미가입, 1: 가입
    'Contract': np.random.choice(['Month-to-month', 'One year', 'Two year'], n)
}

df = pd.DataFrame(data)

# 2. 종속변수(Churn) 생성 (로지스틱 관계)
# col1(요금)이 높을수록, Tenure(기간)가 짧을수록, PhoneService가 없을수록 이탈 확률 높음
# Contract: Month-to-month가 이탈 확률 높음
contract_map = {'Month-to-month': 2, 'One year': 1, 'Two year': 0}
contract_score = df['Contract'].map(contract_map)

logits = -3 + (0.05 * df['col1']) - (0.05 * df['Tenure']) + (0.5 * contract_score) - (1.2 * df['PhoneService'])
probs = 1 / (1 + np.exp(-logits))
df['Churn'] = np.random.binomial(1, probs)

print("데이터 생성 완료!")
print(df.head())

데이터 생성 완료!
   ID       col1  Tenure  PhoneService        Contract  Churn
0   1  67.228295      32             1  Month-to-month      0
1   2  92.028567      17             1  Month-to-month      1
2   3  26.833539      71             1        One year      0
3   4  50.097339      20             1  Month-to-month      0
4   5  66.978157       6             0        Two year      1


In [250]:
# 문제 1번
df_d = df.drop('ID', axis=1)
from statsmodels.formula.api import logit
model = logit('Churn ~ col1 + Tenure + PhoneService + C(Contract)', data=df_d).fit()
print(round(model.pvalues['col1'], 3))

# 문제 2번
odds_ratio = np.exp(model.params['PhoneService'])
print(round(odds_ratio, 3))

# 문제 3번
pred = model.predict(df_d)
result3 = sum(pred >= 0.3)
print(result3)

Optimization terminated successfully.
         Current function value: 0.387853
         Iterations 7
0.0
0.332
128


📝 [제3유형] 유의하지 않은 변수 탐색 (다중회귀)
문제 1. (유의하지 않은 변수 개수) Price를 종속변수로 하고, 나머지 모든 변수(Size, Rooms, Year, Distance, Lucky_Num)를 독립변수로 하는 **다중선형회귀 모델(OLS)**을 적합하시오. 유의수준 0.05 하에서 통계적으로 유의하지 않은(p-value > 0.05) 설명변수의 개수를 구하시오. (단, 절편항(Intercept)은 개수에서 제외하시오.)

문제 2. (가장 유의하지 않은 변수) 위 모델에서 p-value가 가장 큰(가장 유의하지 않은) 변수의 이름을 적으시오.

문제 3. (수정된 결정계수) 위 모델의 설명력을 나타내는 수정된 결정계수(Adjusted R-squared) 값을 구하시오. (단, 소수점 셋째 자리에서 반올림하여 둘째 자리까지 출력하시오.)

In [251]:
import pandas as pd
import numpy as np

np.random.seed(2025)
n = 300

# 독립변수 생성
data = {
    'Size': np.random.normal(30, 10, n),      # 평수
    'Rooms': np.random.randint(1, 6, n),      # 방 개수
    'Year': np.random.randint(1990, 2023, n), # 건축 연도
    'Distance': np.random.normal(5, 2, n),    # 역까지 거리 (가짜 변수)
    'Lucky_Num': np.random.randint(1, 10, n)  # 행운의 숫자 (가짜 변수)
}

df = pd.DataFrame(data)

# 종속변수(Price) 생성
# 가격 = 평수*5 + 방*2 + (연도-1990)*0.5 + 오차
# (Distance와 Lucky_Num은 가격 결정 식에 포함되지 않음 -> 유의하지 않아야 함)
df['Price'] = (5 * df['Size']) + (2 * df['Rooms']) + (0.5 * (df['Year'] - 1990))
df['Price'] += np.random.normal(0, 5, n) # 노이즈 추가

print("데이터 생성 완료!")
print(df.head())

데이터 생성 완료!
        Size  Rooms  Year  Distance  Lucky_Num       Price
0  29.076098      1  2014  3.753131          6  155.242173
1  37.342856      3  1992  8.255590          2  187.675429
2  15.611180      4  2017  3.419036          6  109.319247
3  23.365780      1  2020  8.985747          2  132.536119
4  28.992719      3  2015  3.060076          4  162.630936


In [252]:
# 문제 1번
from statsmodels.formula.api import ols
model = ols('Price ~ Size + Rooms + Year + Distance + Lucky_Num', data=df).fit()
p_val = model.pvalues[1:]
#print(p_val)
count = sum(p_val > 0.05)
print(count)

# 문제 2번
worst_val = p_val.idxmax()
print(worst_val)

# 문제 3번
adj = model.rsquared_adj
print(adj)

1
Distance
0.9901308799657466


📝 [제3유형] F-검정 & 합동분산추정량

문제 1. (F-검정 통계량)두 기계(A, B)의 제품 무게(Weight) 분산이 같은지 확인하기 위해 F-검정을 수행하려고 합니다.**A 기계의 분산($s_A^2$)**과 **B 기계의 분산($s_B^2$)**을 각각 구한 후, F-검정 통계량을 구하시오.(단, F-통계량은 큰 분산을 작은 분산으로 나눈 값으로 정의하며, 분산 계산 시 ddof=1(불편분산)을 사용하시오. 결과는 소수 셋째 자리까지 반올림하여 출력하시오.)

문제 2. (합동분산추정량)위 1번의 결과와 관계없이, 두 기계의 모분산이 같다고 가정(equal_var=True)합니다.이때 두 집단의 분산을 하나로 합친 **합동분산추정량(Pooled Variance, $s_p^2$)**을 구하시오.(단, 반올림하여 소수 셋째 자리까지 출력하시오.)

문제 3. (t-검정)합동분산추정량을 사용하여(즉, 등분산을 가정하고) 두 기계 A, B의 평균 무게에 차이가 있는지 독립표본 t-검정을 수행하시오.이때의 **p-값(p-value)**을 구하시오.(단, 반올림하여 소수 넷째 자리까지 출력하시오.)

In [253]:
import pandas as pd
import numpy as np

np.random.seed(2025)
n = 30 # 표본 크기 30개

# 데이터 생성 (분산이 비슷하게 설정됨)
group_a = np.random.normal(200, 5, n)       # 평균 200, 표준편차 5
group_b = np.random.normal(202, 5.5, n)     # 평균 202, 표준편차 5.5

df = pd.DataFrame({
    'Machine': ['A']*n + ['B']*n,
    'Weight': np.concatenate([group_a, group_b])
})

print("데이터 생성 완료!")
#print(df.groupby('Machine')['Weight'].describe())
print(df.head())

데이터 생성 완료!
  Machine      Weight
0       A  199.538049
1       A  203.671428
2       A  192.805590
3       A  196.682890
4       A  199.496360


In [254]:
# 문제 1번
group_A = df[df['Machine']=='A']['Weight']
group_B = df[df['Machine']=='B']['Weight']

import numpy as np
A_var = np.var(group_A, ddof = 1)
B_var = np.var(group_B, ddof = 1)

if A_var > B_var:
    f_stat = A_var / B_var
else:
    f_stat = B_var / A_var

print(round(f_stat, 3))

# 문제 2번
A_n = len(group_A)
B_n = len(group_B)

pool_var = ((A_n - 1)*A_var + (B_n - 1)*B_var) / ((A_n - 1) + (B_n - 1))
print(round(pool_var, 3))

# 문제 3번
from scipy.stats import ttest_ind
t_stat, p_val = ttest_ind(group_A, group_B, equal_var=True)
print(round(p_val, 4))

1.471
24.078
0.3538
